<a href="https://colab.research.google.com/github/knuffelbeer/leren_en_beslissen_public/blob/main/company_emissions_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import packages
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.neural_network import MLPRegressor

# Import modules

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

#open dataset
df = pd.read_csv("20220105_company_emissions_data.csv", sep=',')

#drop irrelevant columns from dataset
df = df.drop(columns=['WorkforceDataPoints.AsianMinoritiesEmployeesPercent',
                       'WorkforceDataPoints.AsianMinoritiesManagersPercent', 
                      'WorkforceDataPoints.AverageEmployeeLengthOfService',
                      'WorkforceDataPoints.AverageTrainingHours', 
                      'WorkforceDataPoints.BbbeeLevel', 
                      'WorkforceDataPoints.BlackOrAfricanAmericanMinoritiesEmployeesPercent',
                      'WorkforceDataPoints.BlackOrAfricanAmericanMinoritiesManagersPercent',
                      'WorkforceDataPoints.ContractorAccidents',
                      'WorkforceDataPoints.ContractorFatalities',
                      'WorkforceDataPoints.ContractorLostWorkingDays',
                      'WorkforceDataPoints.DayCareServices',
                      'WorkforceDataPoints.DiversityAndOpportunityControversies',
                      'WorkforceDataPoints.EmployeeAccidents',
                      'WorkforceDataPoints.EmployeeFatalities',
                      'WorkforceDataPoints.EmployeeHealthAndSafetyTrainingHours',
                      'WorkforceDataPoints.EmployeeLostWorkingDays',
                      'WorkforceDataPoints.EmployeeResourceGroups',
                      'WorkforceDataPoints.EmployeeSatisfaction',
                      'WorkforceDataPoints.EmployeesHealthAndSafetyControversies',
                      'WorkforceDataPoints.EmployeesHealthAndSafetyOhsas18001',
                      'WorkforceDataPoints.EmployeesHealthAndSafetyTeam',
                      'WorkforceDataPoints.EmployeesWithDisabilities',
                      'WorkforceDataPoints.FlexibleWorkingHours',
                      'WorkforceDataPoints.GenderPayGapPercentage',
                      'WorkforceDataPoints.HealthAndSafetyTraining',
                      'WorkforceDataPoints.HispanicOrLatinoMinoritiesEmployeesPercent',
                      'WorkforceDataPoints.HispanicOrLatinoMinoritiesManagersPercent',
                      'WorkforceDataPoints.HivAidsProgram',
                      'WorkforceDataPoints.HrcCorporateEqualityIndex',
                      'WorkforceDataPoints.HsmsCertifiedPercentage',
                      'WorkforceDataPoints.InternalPromotion',
                      'WorkforceDataPoints.InvoluntaryTurnoverOfEmployees',
                      'WorkforceDataPoints.LostTimeInjuryRateContractors',
                      'WorkforceDataPoints.LostTimeInjuryRateEmployees',
                      'WorkforceDataPoints.LostTimeInjuryRateTotal',
                      'WorkforceDataPoints.LostWorkingDays',
                      'WorkforceDataPoints.ManagementDepartures',
                      'WorkforceDataPoints.ManagementTraining',
                      'WorkforceDataPoints.MinoritiesEmployeesPercent',
                      'WorkforceDataPoints.MinoritiesManagersPercent',
                      'WorkforceDataPoints.MinoritiesSalaryGapPercent',
                      'WorkforceDataPoints.NewWomenEmployees',
                      'WorkforceDataPoints.NumberOfEmployeesFromCsrReporting',
                      'WorkforceDataPoints.OccupationalDiseases',
                      'WorkforceDataPoints.OtherMinoritiesEmployeesPercent',
                      'WorkforceDataPoints.OtherMinoritiesManagersPercent',
                      'WorkforceDataPoints.PolicyCareerDevelopment',
                      'WorkforceDataPoints.PolicyDiversityAndOpportunity',
                      'WorkforceDataPoints.PolicyEmployeeHealthAndSafety',
                      'WorkforceDataPoints.PolicySkillsTraining',
                      'WorkforceDataPoints.PolicySupplyChainHealthAndSafety',
                      'WorkforceDataPoints.RecentDiversityOpportunityControversies',
                      'WorkforceDataPoints.RecentEmployeeHealthAndSafetyControversies',
                      'WorkforceDataPoints.RecentWagesWorkingConditionControversies',
                      'WorkforceDataPoints.SalariesAndWagesFromCsrReporting',
                      'WorkforceDataPoints.Strikes',
                      'WorkforceDataPoints.SupplierESGTraining',
                      'WorkforceDataPoints.SupplyChainHealthAndSafetyImprovements',
                      'WorkforceDataPoints.SupplyChainHealthAndSafetyTraining',
                      'WorkforceDataPoints.TargetsDiversityAndOpportunity',
                      'WorkforceDataPoints.TotalInjuryRateContractors',
                      'WorkforceDataPoints.TotalInjuryRateEmployees',
                      'WorkforceDataPoints.TotalInjuryRateTotal',
                      'WorkforceDataPoints.TradeUnionRepresentation',
                      'WorkforceDataPoints.TrainingCostsTotal',
                      'WorkforceDataPoints.TrainingHoursTotal',
                      'WorkforceDataPoints.TurnoverOfEmployees',
                      'WorkforceDataPoints.VoluntaryTurnoverOfEmployees',
                      'WorkforceDataPoints.WagesWorkingConditionControversiesCount',
                      'WorkforceDataPoints.WhiteMinoritiesEmployeesPercent',
                      'WorkforceDataPoints.WhiteMinoritiesManagersPercent',
                      'WorkforceDataPoints.WomenEmployees',
                      'WorkforceDataPoints.WomenManagers',
                      'WorkforceIndicators.AnnouncedLayOffsToTotalEmployees',
                      'WorkforceIndicators.HealthAndSafetyPolicy',
                      'WorkforceIndicators.InjuriesToMillionHours',
                      'WorkforceIndicators.LostDaysToTotalDays',
                      'WorkforceIndicators.NetEmploymentCreation',
                      'WorkforceIndicators.SalaryGap',
                      'WorkforceIndicators.TrainingAndDevelopmentPolicy',
                      'WorkforceIndicators.TrainingCostsPerEmployee',
                      'WorkforceIndicators.WagesWorkingConditionControversies'],
                      axis=1)

In [10]:
df = df.rename(columns={"GTAP sector": "sector", "EmissionDataPoints.CO2EquivalentsEmissionDirectScope1" :
                                     "target", "StatementDetails.FinancialPeriodFiscalYear" : "year"})


# replace sector with int for use in neural network and remove rows with null sector
#df_sector_int = df2_names_clean
#for i in range(len(df2_names_clean['sector'].value_counts())):
#    df_sector_int = df_sector_int.replace(df2_names_clean['sector'].value_counts().index[i], i + 1)
#
#df_sector_int = df_sector_int[df_sector_int.sector != 1]


#creates small datasets for testing

#df = df_sector_int[["year", "target", "sector"]]

df = df._get_numeric_data()

for column in df:
  if not df[column].isna().values.all():
    print(np.mean(df[column].dropna()))
    df[column] = df[column].fillna(np.mean(df[column].dropna()))
  else:
    df = df.drop(columns=column)

6367.736494372243
2013.636905821613
0.2743061737186715
9.504302417401565
2446815.282648731
0.17651948759881456
967676.9579158932
0.6660302460289315
0.32376078612862524
3686681.76330867
11168702.873887965
668317.8799268794
4207604.379158038
28.88038804301806
2023.8645031156357
0.08388187034080852
76.90127121649934
12021413811.223879
0.13400923919705235
0.33743869209819377
1863312322.168195
0.1888116653039056
0.1439918256130373
2028786.8466288038
780583.7275559163
408.60716731548524
0.006119500626941015
12835862.578200055
20431.40554850457
0.1059581891029824
4097.5537507379595
0.048119378577261375
0.4947591952300483
0.0008330513107537218
14151940.443204591
28882.228269441846
0.17667815637819415
0.3221935800649736
7592.182443348164
0.07691154627090754
808689.6197417751
62.25588668944891
0.4981961254137021
10346630.070351513
216292765.10389742
1173646.568233031
0.0013784973843271296
0.26632480751075727
141.12531969507242
3.1428975473793614
0.1691847494221217
736.1580627841307
138.702118610

In [11]:
df.isna().values.any()

False

In [12]:
target_column = ['target'] 
predictors = list(set(list(df.columns))-set(target_column))
df[predictors] = df[predictors]/df[predictors].max()
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
target,101106.0,3.686682e+06,1.361428e+07,0.0,3.686682e+06,3.686682e+06,3.686682e+06,3.619808e+09


In [13]:
df

,organization_id,year,StatementDetails.FinancialPeriodIsIncomplete,EmissionDataPoints.AccidentalSpills,EmissionDataPoints.AnalyticEstimatesCO2EquivalentsEmissionTotal,EmissionDataPoints.BiodiversityImpactReduction,EmissionDataPoints.CarbonOffsetsCredits,EmissionDataPoints.CementCO2EquivalentsEmission,EmissionDataPoints.ClimateChangeCommercialRisksOpportunities,target,EmissionDataPoints.CO2EquivalentsEmissionIndirectScope3,EmissionDataPoints.CO2EquivalentsEmissionIndirectScope2,EmissionDataPoints.CO2EquivalentsEmissionTotal,EmissionDataPoints.EmissionReductionTargetPercentage,EmissionDataPoints.EmissionReductionTargetYear,EmissionDataPoints.EmissionsTrading,EmissionDataPoints.EmsCertifiedPercent,EmissionDataPoints.EnvironmentalExpenditures,EmissionDataPoints.EnvironmentalInvestmentsInitiatives,EmissionDataPoints.EnvironmentalPartnerships,EmissionDataPoints.EnvironmentalProvisions,EmissionDataPoints.EnvironmentalRestorationInitiatives,EmissionDataPoints.EWasteReduction,EmissionDataPoints.FlaringGases,EmissionDataPoints.HazardousWaste,EmissionDataPoints.InternalCarbonPricePerTonne,EmissionDataPoints.InternalCarbonPricing,EmissionDataPoints.NonHazardousWaste,EmissionDataPoints.NOxEmissions,EmissionDataPoints.NOxSOxEmissionsReduction,EmissionDataPoints.OzoneDepletingSubstances,EmissionDataPoints.ParticulateMatterReduction,EmissionDataPoints.PolicyEmissions,EmissionDataPoints.PolicyNuclearSafety,EmissionDataPoints.SelfReportedEnvironmentalFines,EmissionDataPoints.SOxEmissions,EmissionDataPoints.StaffTransportationImpactReduction,EmissionDataPoints.TargetsEmissions,EmissionDataPoints.VocEmissions,EmissionDataPoints.VocEmissionsReduction,...,EmissionIndicators.TotalWasteToRevenues,EmissionIndicators.VocEmissionsToRevenues,EmissionIndicators.VocOrPmEmissionsReduction,EmissionIndicators.WasteRecycledToTotalWaste,EmissionIndicators.WaterPollutantEmissionsToRevenues,ResourceUseDataPoints.CementEnergyUse,ResourceUseDataPoints.CoalProducedTotal,ResourceUseDataPoints.ElectricityProduced,ResourceUseDataPoints.ElectricityPurchased,ResourceUseDataPoints.EnergyProducedDirect,ResourceUseDataPoints.EnergyPurchasedDirect,ResourceUseDataPoints.EnergyUseTotal,ResourceUseDataPoints.EnvironmentalControversiesCount,ResourceUseDataPoints.EnvironmentalMaterialsSourcing,ResourceUseDataPoints.EnvironmentalSupplyChainManagement,ResourceUseDataPoints.EnvironmentalSupplyChainMonitoring,ResourceUseDataPoints.EnvironmentalSupplyChainPartnershipTermination,ResourceUseDataPoints.EnvironmentManagementTeam,ResourceUseDataPoints.EnvironmentManagementTraining,ResourceUseDataPoints.FreshWaterWithdrawalTotal,ResourceUseDataPoints.GreenBuildings,ResourceUseDataPoints.GridLossPercentage,ResourceUseDataPoints.IndirectEnergyUse,ResourceUseDataPoints.LandEnvironmentalImpactReduction,ResourceUseDataPoints.PolicyEnergyEfficiency,ResourceUseDataPoints.PolicyEnvironmentalSupplyChain,ResourceUseDataPoints.PolicySustainablePackaging,ResourceUseDataPoints.PolicyWaterEfficiency,ResourceUseDataPoints.RecentEnvironmentalControversies,ResourceUseDataPoints.RenewableEnergyProduced,ResourceUseDataPoints.RenewableEnergyPurchased,ResourceUseDataPoints.RenewableEnergyUse,ResourceUseDataPoints.TargetsEnergyEfficiency,ResourceUseDataPoints.TargetsWaterEfficiency,ResourceUseDataPoints.TotalRenewableEnergy,ResourceUseDataPoints.ToxicChemicalsReduction,ResourceUseDataPoints.WaterRecycled,ResourceUseDataPoints.WaterWithdrawalTotal,WorkforceDataPoints.AccidentsTotal,WorkforceDataPoints.AnnouncedLayOffs
53185,0.271092,0.999505,0.0,0.001552,0.000035,0.0,0.003982,0.746671,0.0,3.686682e+06,0.004155,0.005689,0.006975,0.288804,0.98677,0.0,0.769013,0.002307,0.0,0.0,0.002457,0.0,0.0,0.082471,0.002254,0.011674,0.0,0.005104,0.005675,0.0,0.003807,0.0,0.0,0.0,0.001538,0.001553,0.0,0.0,0.016206,0.0,...,0.00021,0.004175,0.204987,0.013981,0.000252,0.004948,0.088149,0.016758,0.000062,0.000295,0.000068,0.003387,0.107014,0.0,0.0,0.994565,0.0,0.0,0.0,0.00007,0.0,0.135759,0.01825,0.0,0.0,0.0,0.0,0.0,0.288722,0.015699

In [14]:
X = df[predictors].values
y = df[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=40)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.10, random_state=40)
print(X_train.shape); print(X_test.shape)

(81895, 95)
(10111, 95)


In [15]:
print(y_train.shape)

(81895, 1)


In [16]:
np.max(X_train)

1.0

In [23]:
"""In this step, we will build the neural network model using the scikit-learn library's estimator object, 'Multi-Layer Perceptron Classifier'. The first line of code (shown below) imports 'MLPClassifier'.

The second line instantiates the model with the 'hidden_layer_sizes' argument set to three layers, which has the same number of neurons as the count of features in the dataset. We will also select 'relu' as the activation function and 'adam' as the solver for weight optimization. To learn more about 'relu' and 'adam', please refer to the Deep Learning with Keras guides, the links of which are given at the end of this guide.

The third line of code fits the model to the training data, while the fourth and fifth lines use the trained model to generate predictions on the training and test dataset, respectively."""

# FEEDBACK, for regression probelem, use regressor, not classifier
from sklearn.neural_network import MLPRegressor

mlp = MLPRegressor(hidden_layer_sizes=(8,8,6,6,2), activation='relu', solver='adam', max_iter=500)
print(y_train.shape)
mlp.fit(X_train, y_train.ravel())


# FEEDBACK, use train, val, test, not just train test
mlp.predict(X_test)

(81895, 1)


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


array([1484771.54672222, 3476232.59480682, 1955544.75072475, ...,
       1967433.24517227, 3841558.5163236 , 3435523.15023736])

In [24]:
# FEEDBACK, for regression problem, use regression metrics e.g. Mean squared error
mlp.score(X_test, y_test)

0.6320015150817605

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators = 500, random_state = 0)
rfr.fit(X_train, y_train.ravel())
rfr.predict(X_test)

In [ ]:
rfr.score(X_test, y_test)